# Setting the fundarmental group of $\Sigma_{g,1}$ 

In [1]:
require './freegroup.rb'

true

## generators

In [2]:
genus = 3
alphabet = %w(a b c d e f)

Gens = [Letter.new] + alphabet.map{|x| Letter.new(x)}
Gens.freeze
p Gens[1..-1].map(&:show)

a = [Gens[0]] + Gens[1..-1].each_slice(2).map(&:first)
b = [Gens[0]] + Gens[1..-1].each_slice(2).map(&:last)
p a.map(&:show)
nil

["a", "b", "c", "d", "e", "f"]
["1", "a", "c", "e"]


In [3]:
comms = []
(Gens.size/2 + 1).times do |i|
  comms << Group.commutator(a[i], b[i])
end
p comms.map(&:show)

["1111", "abAB", "cdCD", "efEF"]


["1111", "abAB", "cdCD", "efEF"]

In [4]:
Del = (comms[1]*comms[2]*comms[3]*Word.new('s')).freeze
p 'Del = ' + Del.show

DelPerms = []
perm = Del.dup
(Del.size).times do |n|
  perm = perm.cyclic_permutation
  DelPerms << perm << perm.inverse
end
DelPerms.freeze
p DelPerms.map(&:show)
nil

"Del = (((abAB)(cdCD))(efEF))(s)"
["bABcdCDefEFsa", "ASfeFEdcDCbaB", "ABcdCDefEFsab", "BASfeFEdcDCba", "BcdCDefEFsabA", "aBASfeFEdcDCb", "cdCDefEFsabAB", "baBASfeFEdcDC", "dCDefEFsabABc", "CbaBASfeFEdcD", "CDefEFsabABcd", "DCbaBASfeFEdc", "DefEFsabABcdC", "cDCbaBASfeFEd", "efEFsabABcdCD", "dcDCbaBASfeFE", "fEFsabABcdCDe", "EdcDCbaBASfeF", "EFsabABcdCDef", "FEdcDCbaBASfe", "FsabABcdCDefE", "eFEdcDCbaBASf", "sabABcdCDefEF", "feFEdcDCbaBAS", "abABcdCDefEFs", "SfeFEdcDCbaBA"]


In [5]:
Rseq = []
(Gens.size/2).times do |i|
  Rseq += [a[i+1], b[i+1].inverse, a[i+1].inverse, b[i+1]]
end
Rseq.freeze
p Rseq.map(&:show) #.index('A')
nil

<main>:2: warning: already initialized constant Rseq
<main>:2: warning: already initialized constant Rseq
<main>:2: warning: previous definition of Rseq was here
<main>:2: warning: already initialized constant Rseq
<main>:2: warning: previous definition of Rseq was here


["a", "B", "A", "b", "c", "D", "C", "d", "e", "F", "E", "f"]


# The Algorithm

In [33]:
w = Word.new('abDd')
w = w*DelPerms[5].split(9)[0]
p w.show
nil

"(abDd)(aBASfeFEd)"


### (i) Contracting the word

In [34]:
# (i) contract
reduced = w.contract.dup
reduced.show

"abaBASfeFEd"

### (ii) removing subwords

In [8]:
def reduce_once(myw)
  facs = myw.flatten.factors
  (myw.size+1).times do |i|
    w1 = Word.new(facs[0,i])
    w2 = Word.new(facs.drop(i))
    original_size = myw.size
    for perm in DelPerms
      wc = (w1*perm*w2).deep_copy.contract
      if (wc.size < original_size) then
        myw = wc 
        break
      end
    end
    if (myw.size < original_size) then
      return myw
      break
    end
  end
  return nil    
end
    
def reduce(myword)
  while not myword.nil?
    prev = myword
    myword = reduce_once(myword)
  end
  return prev.contract
end
nil

In [35]:
p 'w = ' + w.show + ' and w.size = ' + w.size.to_s
p reduce(w).show

"w = abaBASfeFEd and w.size = 11"
"acdC"


"acdC"

### (iii) removing subwords part2

In [10]:
# not necessary?

## The algorithm

In [11]:
gens_ex = Gens + [Letter.new('s')]
for g in gens_ex[1..-1]
  p g.show + ': ' + w.count(g).to_s
  p g.inverse.show + ': ' + w.count(g.inverse).to_s
end;nil

"a: 1"
"A: 1"
"b: 2"
"B: 1"
"c: 2"
"C: 0"
"d: 2"
"D: 0"
"e: 0"
"E: 0"
"f: 0"
"F: 0"
"s: 0"
"S: 0"


In [12]:
def N(w, gens)
  gens.map{|g| w.count(g)}.inject(0, &:+)
end;nil

In [13]:
def P(w)
  count = 0
  facs = w.flatten.factors
  facs.each_with_index do |f, i|
    succ = (i < facs.size - 1) ? facs[i+1] : facs[0]
    pair = [f.inverse, succ]
    # p pair.map(&:show)
    idxs = pair.map{|f| Rseq.find_index(f)}
    # p idxs
    count += 1 if idxs[0] < idxs[1]
  end
  return count
end

:P

In [14]:
myw = Word.new(a[1], b[2].inverse).contract
subs = myw.split(1)
ww = subs[0]*(subs[1].inverse)
p "w = #{myw.show}, div = #{subs.map(&:show)}, ww = #{ww.show}"

ww.flatten!
nil

"w = aD, div = [\"a\", \"D\"], ww = (a)(d)"


In [15]:
def T(w, verbose=false)
  w.flatten!
  ais = Gens[1..-1].each_slice(2).map(&:first).map!(&:inverse)
  bs = Gens[1..-1].each_slice(2).map(&:last)
  si = [Letter.new('S')]
  
  result = P(w)-N(w, ais+bs+si)
  return (!verbose) ? result : result.to_s + "  [P=#{P(w)}, A_bar=#{N(w, ais)}, B=#{N(w, bs)}, S_bar=#{N(w, si)}]"
end

:T

In [16]:
myw = Word.new(a[1], b[2].inverse).contract
subs = myw.split(1)
ww = subs[0]*(subs[1].inverse)
p "w = #{myw.show}, div = #{subs.map(&:show)}, ww = #{ww.show}"

p "T(w) = #{T(ww, true)}"
nil

"w = aD, div = [\"a\", \"D\"], ww = (a)(d)"
"T(w) = 0  [P=1, A_bar=0, B=1, S_bar=0]"


In [17]:
w = Word.new(a[1], b[1], a[1].inverse)
p w.show
p w.first == w.last.inverse
w.factors[1..-2].map(&:show)
w.cyclic_permutation(1).contract.show
w.size

"abA"
true


3

In [110]:
100.times do |e|
  rw = Word.new
  10.times do |c|
    g = (rand(2)==0) ? Gens[rand(1...6)] : Gens[rand(1...6)].inverse
    rw = (rw*Word.new(g)).contract
  end
  rw.factors.delete(Letter.new)
  num = P(rw)+P(rw.inverse)
  if num != rw.size 
    p "word = #{rw.show}, size = #{rw.size}, num = #{num}"
  end
end; nil

"word = dcDAeBDD, size = 8, num = 7"
"word = DEcBad, size = 6, num = 5"
"word = ebeABAeaaE, size = 10, num = 9"
"word = CCaddc, size = 6, num = 5"
"word = dBeBADcD, size = 8, num = 7"
"word = BDcAABDb, size = 8, num = 7"
"word = bECAdcdB, size = 8, num = 7"
"word = AdcbdcdcBa, size = 10, num = 9"
"word = ebbEdE, size = 6, num = 5"
"word = dBaBaddCED, size = 10, num = 9"
"word = eCAeaDCE, size = 8, num = 7"
"word = dbAEEcBDDD, size = 10, num = 9"
"word = cEdBeDaabC, size = 10, num = 9"
"word = AcbDcdca, size = 8, num = 7"
"word = DEDead, size = 6, num = 5"
"word = AdBDBBDEda, size = 10, num = 9"
"word = caBeAC, size = 6, num = 5"
"word = CAdeDBBc, size = 8, num = 7"


In [342]:
def examine(myw, verbose=false)
  flag = 0
  result = ''
  myw = myw.contract
  (1...myw.size).each do |i|
    div = myw.split(i)
    words = [div[0]*(div[1].inverse), div[0].dup, div[1].inverse.dup]
    for w in words
      wcp = Group::Identity
      while wcp.size < w.size
        w.factors = wcp.factors unless wcp == Group::Identity
        wcp = w.cyclic_permutation.contract
      end
    end
    v0, v1, v2 = T(words[0]), T(words[1]), T(words[2])
    if (verbose or v0 != v1 + v2) then
      flag = 1
      result += "#{div.map(&:show)}  --->  " + \
        "T(#{words[0].show}) = #{v0}, T(#{words[1]}) + T(#{words[2]}) = #{v1} + #{v2} = #{v1+v2}" + \
        "\n"
    end
  end
  printf (flag==0) ? "*** SIMPLE ***" : result
end; nil

In [343]:
#myw = Word.new(a[1])*(comms[2].inverse)
myw = Word.new(a[1])*comms[2]
p "w = #{myw.show}"
examine(myw); nil

"w = (a)(cdCD)"
*** SIMPLE ***

In [331]:
w1 = Word.new(b[2].inverse, b[1].inverse).contract
w2 = Word.new(a[2], a[1]).contract

myw = w1*w2*(w1.inverse)*(w2.inverse)

p "myw = #{myw.show}"
T(myw, true)

"myw = (((DB)(ca))(bd))(AC)"


"1  [P=5, A_bar=2, B=2, S_bar=0]"

## Experiments

### the words $a_{1}([a_{2},b_{2}]^{c})^{k}$, where $c=b_{1}^{a_{1}}$ and $k \in \mathbb{Z}$

In [346]:
a1 = Word.new(a[1])
k=2

#-----------------
myw = a1; k.times{|i| myw = myw*comms[2]}
p "myw = #{myw.show}"
examine(myw)

#-----------------
c = Word.new(a[1], b[1], a[1].inverse)
gamma = Group.commutator(comms[2].inverse, c)
#p (comms[2]*gamma).contract.show

ow = a1; k.times{|i| ow = ow*((comms[2]*gamma).contract)}
p "ow = #{ow.show}, #{ow.contract.show}"
examine(ow.contract, 1); nil

"myw = ((a)(cdCD))(cdCD)"
["acdCD", "cdCD"]  --->  T(a) = 0, T(acdCD) + T(dcDC) = 2 + -1 = 1
"ow = ((a)(abAcdCDaBA))(abAcdCDaBA), aabAcdCDcdCDaBA"
["a", "abAcdCDcdCDaBA"]  --->  T(abAdcDCdcDCaB) = -2, T(a) + T(dcDCdcDC) = 0 + -2 = -2
["aa", "bAcdCDcdCDaBA"]  --->  T(aaabAdcDCdcDCaB) = -2, T(aa) + T(abAdcDCdcDCaB) = 0 + -2 = -2
["aab", "AcdCDcdCDaBA"]  --->  T(aababAdcDCdcDCa) = -2, T(aab) + T(abAdcDCdcDCa) = 0 + -2 = -2
["aabA", "cdCDcdCDaBA"]  --->  T(abbAdcDCdcDCa) = -2, T(ab) + T(abAdcDCdcDC) = 0 + -2 = -2
["aabAc", "dCDcdCDaBA"]  --->  T(aabAcabAdcDCdcD) = 0, T(aabAc) + T(abAdcDCdcD) = 1 + -1 = 0
["aabAcd", "CDcdCDaBA"]  --->  T(aabAcdabAdcDCdc) = 0, T(aabAcd) + T(abAdcDCdc) = 1 + -1 = 0
["aabAcdC", "DcdCDaBA"]  --->  T(aabAcdCabAdcDCd) = 0, T(aabAcdC) + T(abAdcDCd) = 1 + -1 = 0
["aabAcdCD", "cdCDaBA"]  --->  T(aabAcdCDabAdcDC) = 1, T(aabAcdCD) + T(abAdcDC) = 2 + -1 = 1
["aabAcdCDc", "dCDaBA"]  --->  T(aabAcdCDcabAdcD) = 2, T(aabAcdCDc) + T(abAdcD) = 2 + 0 = 2
["aabAcdCDcd", "CDaBA

### the word $a_{1}[a_{2}, a_{3}]$

In [348]:
myw = (Word.new(a[1])*Group.commutator(a[2], a[3])).contract
p "w = #{myw.show}"

examine(myw)

"w = aceCE"
["ac", "eCE"]  --->  T(acecE) = 2, T(ac) + T(c) = 1 + 0 = 1
["ace", "CE"]  --->  T(aceec) = 2, T(ace) + T(ec) = 2 + 1 = 3
["aceC", "E"]  --->  T(aceCe) = 2, T(aceC) + T(e) = 1 + 0 = 1


Mod $\Gamma_{3}$

In [349]:
div = myw.split(4)
p div.map(&:show)
gamma = Group.commutator(a[1], comms[1])

ow = div[0]*(gamma*div[1]*gamma.inverse)
p ow.show

examine(ow)

["aceC", "E"]
"(aceC)(((aabABAbaBA)(E))(abABabaBAA))"
*** SIMPLE ***

In [380]:
(myw.size-1).times do |i|
  div = myw.split(i+1)
  p div.map(&:show)

  gamma = Group.commutator(a[3], comms[3])
  ow = div[0]*(gamma*div[1]*(gamma.inverse))
  p ow.show

  '''
  wcp = Group::Identity
  while wcp.size < ow.size
    ow.factors = wcp.factors unless wcp == Group::Identity
    wcp = ow.cyclic_permutation.contract
  end
  p ow.show
  '''
  a1 = Word.new(a[1])
  ow = ((a1.inverse)*ow*a1).contract
  p ow.show
  
  examine(ow)
  printf "\n\n"
end; nil

["a", "ceCE"]
"(a)(((eefEFEfeFE)(ceCE))(efEFefeFEE))"
"eefEFEfeFEceCfEFefeFEEa"
["e", "efEFEfeFEceCfEFefeFEEa"]  --->  T(AeefEFEfeFcECefEFefeF) = 1, T(e) + T(AeefEFEfeFcECefEFefeFE) = 0 + 0 = 0
["eefEFE", "feFEceCfEFefeFEEa"]  --->  T(eefEFEAeefEFEfeFcECefEF) = 1, T(efEF) + T(AeefEFEfeFcECefEF) = 1 + 1 = 2
["eefEFEfeFEce", "CfEFefeFEEa"]  --->  T(eefEFEfeFEceAeefEFEfeFc) = 1, T(eefEFEfeFEce) + T(AeefEFEfeFc) = 0 + 0 = 0
["eefEFEfeFEceC", "fEFefeFEEa"]  --->  T(eefEFEfeFEceCAeefEFEfeF) = -1, T(eefEFEfeFEceC) + T(AeefEFEfeF) = 0 + 0 = 0
["eefEFEfeFEceCfEF", "efeFEEa"]  --->  T(efEFEfeFEceCfEFAeefEF) = 1, T(eefEFEfeFEceCfEF) + T(AeefEFE) = 0 + 0 = 0
["eefEFEfeFEceCfEFefeFE", "Ea"]  --->  T(eefEFEfeFEceCfEFefeFEAe) = -1, T(efEFEfeFEceCfEFefeF) + T(Ae) = 0 + 0 = 0
["eefEFEfeFEceCfEFefeFEE", "a"]  --->  T(eefEFEfeFEceCfEFefeFEEA) = -1, T(EceC) + T(A) = 0 + 0 = 0


["ac", "eCE"]
"(ac)(((eefEFEfeFE)(eCE))(efEFefeFEE))"
"ceefEFEfeFCfEFefeFEEa"
*** SIMPLE ***

["ace", "CE"]
"(ace)(((eefEFEfeFE)(

In [377]:
div = myw.split(2)
p div.map(&:show)

gamma = Group.commutator(a[3], comms[3])
ow = div[0]*(gamma*div[1]*(gamma.inverse))
p ow.show
p ow.contract.show

examine(ow)

["ac", "eCE"]
"(ac)(((eefEFEfeFE)(eCE))(efEFefeFEE))"
"aceefEFEfeFCfEFefeFEE"
["ac", "eefEFEfeFCfEFefeFEE"]  --->  T(aceefEFEfeFcfEFefeFEE) = 2, T(ac) + T(c) = 1 + 0 = 1
["ace", "efEFEfeFCfEFefeFEE"]  --->  T(aceeefEFEfeFcfEFefeFE) = 2, T(ace) + T(efEFEfeFcfEFefeF) = 2 + 1 = 3
["aceefEFE", "feFCfEFefeFEE"]  --->  T(ceefEFefEFEfeFcfEFa) = 4, T(aceefEFE) + T(eefEFEfeFcfEF) = 2 + 1 = 3
["aceefEFEfeF", "CfEFefeFEE"]  --->  T(aceefEFEfeFeefEFEfeFc) = 2, T(aceefEFEfeF) + T(eefEFEfeFc) = 2 + 1 = 3
["aceefEFEfeFC", "fEFefeFEE"]  --->  T(aceefEFEfeFCeefEFEfeF) = 2, T(aceefEFEfeFC) + T(eefEFEfeF) = 1 + 0 = 1
["aceefEFEfeFCfEF", "efeFEE"]  --->  T(aceefEFEfeFCfEFeefEFE) = 2, T(aceefEFEfeFCfEF) + T(efEF) = 2 + 1 = 3
["aceefEFEfeFCfEFefeFE", "E"]  --->  T(ceefEFEfeFCfEFefeFa) = 2, T(aceefEFEfeFCfEFefeFE) + T(e) = 1 + 0 = 1


Mod $\Gamma_{2}$

In [354]:
(myw.size-1).times do |i|
  div = myw.split(i+1)
  p div.map(&:show)

  gamma = comms[1]
  ow = div[0]*(gamma*div[1]*(gamma.inverse))
  p ow.show

  wcp = Group::Identity
  while wcp.size < ow.size
    ow.factors = wcp.factors unless wcp == Group::Identity
    wcp = ow.cyclic_permutation.contract
  end
  p ow.show
  
  examine(ow)
  printf "\n\n"
end; nil

["a", "ceCE"]
"(a)(((abAB)(ceCE))(baBA))"
"abABceCEbaB"
*** SIMPLE ***

["ac", "eCE"]
"(ac)(((abAB)(eCE))(baBA))"
"cabABeCEbaB"
["ca", "bABeCEbaB"]  --->  T(cabABecEbaB) = 2, T(ca) + T(c) = 1 + 0 = 1
["cabAB", "eCEbaB"]  --->  T(abAABecEc) = 2, T(cabAB) + T(bABecE) = 2 + 1 = 3
["cabABeC", "EbaB"]  --->  T(cabABeCbABe) = 2, T(abABe) + T(bABe) = 2 + 1 = 3
["cabABeCE", "baB"]  --->  T(cabABeCEbAB) = 2, T(cabABeCE) + T(A) = 1 + 0 = 1


["ace", "CE"]
"(ace)(((abAB)(CE))(baBA))"
"ceabABCEbaB"
["ceabAB", "CEbaB"]  --->  T(eabAABecc) = 3, T(ceabAB) + T(bABec) = 3 + 1 = 4
["ceabABCE", "baB"]  --->  T(ceabABCEbAB) = 3, T(ceabABCE) + T(A) = 2 + 0 = 2


["aceC", "E"]
"(aceC)(((abAB)(E))(baBA))"
"ceCabABEbaB"
["ceCa", "bABEbaB"]  --->  T(ceCabABebaB) = 2, T(ceCa) + T(e) = 1 + 0 = 1
["ceCabAB", "EbaB"]  --->  T(eCabAABec) = 2, T(ceCabAB) + T(bABe) = 2 + 1 = 3
["ceCabABE", "baB"]  --->  T(ceCabABEbAB) = 2, T(ceCabABE) + T(A) = 1 + 0 = 1




In [234]:
p ow.show
div = ow.split(1)
p "#{div.map(&:show)}"

words = [div[0]*(div[1].inverse), div[0].dup, div[1].inverse.dup]
for w in words
  wcp = Group::Identity
  while wcp.size < w.size
    w.factors = wcp.factors unless wcp == Group::Identity
    wcp = w.cyclic_permutation.contract
  end
end

3.times do |i|
  printf("%-16s = %-30s\n", "T(#{words[i].show})", T(words[i], true))
end; nil

"aceabABCEbaBA"
"[\"a\", \"ceabABCEbaBA\"]"
T((a)(abABecbaBAEC)) = -1  [P=5, A_bar=4, B=2, S_bar=0]
T(a)             = 0  [P=0, A_bar=0, B=0, S_bar=0]
T(abABecbaBAEC)  = -1  [P=5, A_bar=4, B=2, S_bar=0]


In [230]:
p Rseq.map(&:show); nil

["a", "B", "A", "b", "c", "D", "C", "d", "e", "F", "E", "f"]
